In [ ]:
import scipy.io as sio
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np


#load benchmark dataset
N_data = sio.loadmat('PDNA-224-ONEHOT-11-N.mat')
P_data = sio.loadmat('PDNA-224-ONEHOT-11-P.mat')
n_X = N_data['n_data']
n_Y = N_data['n_target']
print(len(n_Y))
print('')

p_X = P_data['p_data']
p_Y = P_data['p_target']
print(len(p_Y))

# X = n_X[0:3778]+p_X
# Y = n_Y[0:3778]+p_Y

X = np.vstack((n_X[0:3778],p_X)) #合并正负样本
Y = np.vstack((n_Y[0:3778],p_Y))

X = X.reshape(7556,-1)
Y = Y.reshape(7556,-1) 
print(X)
#loo = LeaveOneOut()
kf = KFold(n_splits=5)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    print("len(X_train)={},len(Y_test)={}".format(len(X_train),len(Y_test)))
    #cnn(X_train,X_test,Y_train,Y_test,100)


In [9]:
import scipy.io as sio
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np

names = locals()
#load benchmark dataset
N_data = sio.loadmat('PDNA-224-ONEHOT-11-N.mat')
P_data = sio.loadmat('PDNA-224-ONEHOT-11-P.mat')
n_X = N_data['n_data']
n_Y = N_data['n_target']
#print(len(n_Y))
print('')

p_X = P_data['p_data']
p_Y = P_data['p_target']

NN = len(n_X)
#print(NN)

#负样本批次大小
N_size = 3822
#计算一共有负样本多少个批次
NN_batch = NN // N_size

#print(NN_batch)
a=0
#得到14个数据集，正样本不变，负样本从'PDNA-224-ONEHOT-11-N.mat中抽出3822个不放回
for i in range(NN_batch):
        
        
        if (i+2)*N_size > NN: #  53570/3822=14.01 
                X = np.vstack((n_X[i*N_size:NN],p_X))
                Y = np.vstack((n_Y[i*N_size:NN],p_Y))
                #print('i*N_size')
        else:
                X = np.vstack((n_X[i*N_size:(i+1)*N_size],p_X))
                Y = np.vstack((n_Y[i*N_size:(i+1)*N_size],p_Y))
                #print(i*N_size)
                
        #print(len(X),len(Y))
        print('')
        
        X = X.reshape(len(X),-1)
        Y = Y.reshape(len(Y),-1) 
        #print(X)
        #loo = LeaveOneOut()
        kf = KFold(n_splits=5)
        kf.get_n_splits(X)
        for train_index, test_index in kf.split(X):
            
            #得到70个数据集，分别为
            names['X_train_%s' % a],names['X_test_%s' % a]=X[train_index], X[test_index]
            names['Y_train_%s' % a],names['Y_test_%s' % a]=Y[train_index], Y[test_index]
            a+=1
#             X_train, X_test = X[train_index], X[test_index]
#             Y_train, Y_test = Y[train_index], Y[test_index]

print(Y_test_69)
















[[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PDNA-迁移学习-测试集未分开

"""

import scipy.io as sio
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np



#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #input tensor of shape [batch, in_height, in_width, in_channels]
    #filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0]=strides[3]=1. strides[1]代表ｘ方向的步长，strids[2]代表ｙ方向的步长
    #padding: A string from "SAME", "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def cnn():
 


    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 460])#28*28
    y = tf.placeholder(tf.float32, [None, 2])

    #改变x的格式转为４Ｄ的向量【batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x,[-1, 23, 20 ,1])

    #初始化第一个卷积层的权值和偏量
    W_conv1 = weight_variable([5,5,1,32])#5*5的采样窗口，３２个卷积核从１个平面抽取特征
    b_conv1 = bias_variable([32])#每一个卷积核一个偏置值

    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

    #初始化第二个卷积层的权值和偏置
    W_conv2 = weight_variable([5,5,32,64]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
    b_conv2 = bias_variable([64]) #每一个卷积核一个偏置值

    #把H_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    #23*20的图片第一次卷积后还是23*20,第一次池化后变为12*10
    #第二次卷积后为12*10,第二次池化后变为6*5
    #进过上面操作后得到64张6*5的平面

    #初始化第一全链接层的权值
    W_fc1 = weight_variable([6*5*64,1024]) #上一层有7*7*64个神经元,全连接层有1024个神经元
    b_fc1 = bias_variable([1024])

    #把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool2,[-1,6*5*64])
    #求第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #keep_prob用了表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

    #初始化第二个全连接层
    W_fc2 = weight_variable([1024,2])
    b_fc2 = bias_variable([2])

    #计算输出
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

    #交叉熵代价函数
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    #使用AdamOptimizer进行优化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    #结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    #求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(5):
            for batch in range(n_batch):
                #训练模型

                if (batch+1)*batch_size > N:
                        batch_xs = X_train[batch*batch_size:N]
                        batch_ys = Y_train[batch*batch_size:N]
                else:
                        batch_xs = X_train[batch*batch_size:(batch+1)*batch_size]
                        batch_ys = Y_train[batch*batch_size:(batch+1)*batch_size]
                        #batch_xs,batch_ys = mnist.train.next_batch(batch_size)
                sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})

            
            acc = sess.run(accuracy, feed_dict={x:names['X_test_%s' % i], y:names['Y_test_%s' % i], keep_prob:1.0})
            print("Iter " + str(epoch) + "，Testing Accuracy=" + str(acc))
            #print('循环'+str(epoch)+"，Iter " + str(i) + "，Testing Accuracy=" + str(acc))
            
            


names = locals()
#load benchmark dataset
N_data = sio.loadmat('PDNA-224-ONEHOT-11-N.mat')
P_data = sio.loadmat('PDNA-224-ONEHOT-11-P.mat')
n_X = N_data['n_data']
n_Y = N_data['n_target']
#print(len(n_Y))
print('')

p_X = P_data['p_data']
p_Y = P_data['p_target']

NN = len(n_X)
#print(NN)

#负样本批次大小
N_size = 3822
#计算一共有负样本多少个批次
NN_batch = NN // N_size

#print(NN_batch)
a=0
#得到14个数据集，正样本不变，负样本从'PDNA-224-ONEHOT-11-N.mat中抽出3822个不放回
for i in range(NN_batch):
        
        
        if (i+2)*N_size > NN: #  53570/3822=14.01 
                X = np.vstack((n_X[i*N_size:NN],p_X))
                Y = np.vstack((n_Y[i*N_size:NN],p_Y))
                #print('i*N_size')
        else:
                X = np.vstack((n_X[i*N_size:(i+1)*N_size],p_X))
                Y = np.vstack((n_Y[i*N_size:(i+1)*N_size],p_Y))
                #print(i*N_size)
                
        #print(len(X),len(Y))
        #print('')
        
        X = X.reshape(len(X),-1)
        Y = Y.reshape(len(Y),-1) 
        #print(X)
        #loo = LeaveOneOut()
        kf = KFold(n_splits=5)
        kf.get_n_splits(X)
        for train_index, test_index in kf.split(X):
            
            #得到70个数据集，保存在动态变量中,例如 X_train_1
            names['X_train_%s' % a],names['X_test_%s' % a]=X[train_index], X[test_index]
            names['Y_train_%s' % a],names['Y_test_%s' % a]=Y[train_index], Y[test_index]
            a+=1
            
cnn()            






Iter 0，Testing Accuracy=0.55874676
Iter 1，Testing Accuracy=0.5848564
Iter 2，Testing Accuracy=0.62663186
Iter 3，Testing Accuracy=0.6285901
Iter 4，Testing Accuracy=0.6462141
Iter 5，Testing Accuracy=0.6821149
Iter 6，Testing Accuracy=0.6932115
Iter 7，Testing Accuracy=0.7160574
Iter 8，Testing Accuracy=0.7389034
Iter 9，Testing Accuracy=0.7578329
Iter 10，Testing Accuracy=0.78067887
Iter 11，Testing Accuracy=0.7983029
Iter 12，Testing Accuracy=0.8172324
Iter 13，Testing Accuracy=0.82832897
Iter 14，Testing Accuracy=0.8302872
Iter 15，Testing Accuracy=0.8348564
Iter 16，Testing Accuracy=0.84856397
Iter 17，Testing Accuracy=0.8557441
Iter 18，Testing Accuracy=0.8616188
Iter 19，Testing Accuracy=0.883812
Iter 20，Testing Accuracy=0.88577026
Iter 21，Testing Accuracy=0.89033943
Iter 22，Testing Accuracy=0.9086162
Iter 23，Testing Accuracy=0.9144909
Iter 24，Testing Accuracy=0.91644907
Iter 25，Testing Accuracy=0.92493474
Iter 26，Testing Accuracy=0.9295039
Iter 27，Testing Accuracy=0.93929505
Iter 28，Testing Accu

In [ ]:



Iter  str(epoch)Testing Accuracy=0.8922976



Iter 0，Testing Accuracy=0.55874676
Iter 1，Testing Accuracy=0.5848564
Iter 2，Testing Accuracy=0.62663186
Iter 3，Testing Accuracy=0.6285901
Iter 4，Testing Accuracy=0.6462141
Iter 5，Testing Accuracy=0.6821149
Iter 6，Testing Accuracy=0.6932115
Iter 7，Testing Accuracy=0.7160574
Iter 8，Testing Accuracy=0.7389034
Iter 9，Testing Accuracy=0.7578329
Iter 10，Testing Accuracy=0.78067887
Iter 11，Testing Accuracy=0.7983029
Iter 12，Testing Accuracy=0.8172324
Iter 13，Testing Accuracy=0.82832897
Iter 14，Testing Accuracy=0.8302872
Iter 15，Testing Accuracy=0.8348564
Iter 16，Testing Accuracy=0.84856397
Iter 17，Testing Accuracy=0.8557441
Iter 18，Testing Accuracy=0.8616188
Iter 19，Testing Accuracy=0.883812
Iter 20，Testing Accuracy=0.88577026
Iter 21，Testing Accuracy=0.89033943
Iter 22，Testing Accuracy=0.9086162
Iter 23，Testing Accuracy=0.9144909
Iter 24，Testing Accuracy=0.91644907
Iter 25，Testing Accuracy=0.92493474
Iter 26，Testing Accuracy=0.9295039
Iter 27，Testing Accuracy=0.93929505
Iter 28，Testing Accuracy=0.94712794
Iter 29，Testing Accuracy=0.9497389
Iter 30，Testing Accuracy=0.95300263
Iter 31，Testing Accuracy=0.95365536
Iter 32，Testing Accuracy=0.95626634
Iter 33，Testing Accuracy=0.95822453
Iter 34，Testing Accuracy=0.95822453
Iter 35，Testing Accuracy=0.95887727
Iter 36，Testing Accuracy=0.95953006
Iter 37，Testing Accuracy=0.962141
Iter 38，Testing Accuracy=0.9627937
Iter 39，Testing Accuracy=0.9640992
Iter 40，Testing Accuracy=0.96475196
Iter 41，Testing Accuracy=0.96475196
Iter 42，Testing Accuracy=0.96475196
Iter 43，Testing Accuracy=0.9660574
Iter 44，Testing Accuracy=0.9680157
Iter 45，Testing Accuracy=0.96932113
Iter 46，Testing Accuracy=0.96932113
Iter 47，Testing Accuracy=0.97062665
Iter 48，Testing Accuracy=0.9712794
Iter 49，Testing Accuracy=0.9712794
Iter 50，Testing Accuracy=0.9719321
Iter 51，Testing Accuracy=0.9719321
Iter 52，Testing Accuracy=0.9719321
Iter 53，Testing Accuracy=0.9719321
Iter 54，Testing Accuracy=0.97258484
Iter 55，Testing Accuracy=0.97258484
Iter 56，Testing Accuracy=0.97258484
Iter 57，Testing Accuracy=0.97258484
Iter 58，Testing Accuracy=0.97258484
Iter 59，Testing Accuracy=0.97258484
Iter 60，Testing Accuracy=0.9732376
Iter 61，Testing Accuracy=0.97389036
Iter 62，Testing Accuracy=0.97389036
Iter 63，Testing Accuracy=0.9745431
Iter 64，Testing Accuracy=0.9751958
Iter 65，Testing Accuracy=0.97584856
Iter 66，Testing Accuracy=0.97584856
Iter 67，Testing Accuracy=0.97584856
Iter 68，Testing Accuracy=0.97584856
Iter 69，Testing Accuracy=0.97584856
Iter 70，Testing Accuracy=0.97584856
Iter 71，Testing Accuracy=0.9765013
Iter 72，Testing Accuracy=0.9765013
Iter 73，Testing Accuracy=0.9765013
Iter 74，Testing Accuracy=0.9765013
Iter 75，Testing Accuracy=0.9765013
Iter 76，Testing Accuracy=0.9771541
Iter 77，Testing Accuracy=0.9771541
Iter 78，Testing Accuracy=0.9771541
Iter 79，Testing Accuracy=0.9771541
Iter 80，Testing Accuracy=0.9771541
Iter 81，Testing Accuracy=0.9771541
Iter 82，Testing Accuracy=0.9771541
Iter 83，Testing Accuracy=0.9771541
Iter 84，Testing Accuracy=0.9778068
Iter 85，Testing Accuracy=0.9778068
Iter 86，Testing Accuracy=0.9778068
Iter 87，Testing Accuracy=0.9778068
Iter 88，Testing Accuracy=0.9778068
Iter 89，Testing Accuracy=0.9778068
Iter 90，Testing Accuracy=0.9778068
Iter 91，Testing Accuracy=0.9778068
Iter 92，Testing Accuracy=0.9778068
Iter 93，Testing Accuracy=0.9778068
Iter 94，Testing Accuracy=0.9778068
Iter 95，Testing Accuracy=0.9778068
Iter 96，Testing Accuracy=0.9778068
Iter 97，Testing Accuracy=0.9778068
Iter 98，Testing Accuracy=0.9778068
Iter 99，Testing Accuracy=0.9778068




In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  4 11:40:07 2018

@author: weizhong
"""
#数据集 PDNA-224-PSSM-Norm-11.mat

import scipy.io as sio
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

#数据集相关常数
DATA_SIZE = 57348
INPUT_NODE = 460
OUTPUT_NODE = 2
X_SIZE = 23
Y_SIZE = 20
NUM_CHANNELS = 1
NUM_LABELS = 2

#配置神经网络的参数
#第一层卷积层的尺寸和深度
CONV1_DEEP = 32
CONV1_SIZE = 5
#第二层卷积层的尺寸和深度
CONV2_DEEP = 64
CONV2_SIZE = 5

FC_SIZE = 1024 #全连接层的节点个数

BATCH_SIZE = 50 #

LEARNING_RATE = 1e-4 #基础学习率

LEANING_RATE_DECAY = 0.99 #学习率的衰减率

TRAINING_STEPS= 2000 #训练轮数

#不同类的惩罚系数
LOSS_COEF = [100, 1]

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=1,seed=1) #生成一个截断的正态分布
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#初始化偏置
def bias_variable(shape):
    initial = tf.truncated_normal(shape,stddev=1,seed=1)
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#卷积层
def conv2d(x,W):
    #input tensor of shape [batch, in_height, in_width, in_channels]
    #filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0]=strides[3]=1. strides[1]代表ｘ方向的步长，strids[2]代表ｙ方向的步长
    #padding: A string from "SAME", "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def cnn(x_train, x_test, y_train, y_test):
    x_train_size = np.size(x_train, 0)
    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None, INPUT_NODE])#23*20
    y = tf.placeholder(tf.float32, [None, OUTPUT_NODE])

    #改变x的格式转为４Ｄ的向量【batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x,[-1, X_SIZE, Y_SIZE, 1])

    #初始化第一个卷积层的权值和偏量
    W_conv1 = weight_variable([CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP])#5*5的采样窗口，３２个卷积核从4个平面抽取特征
    b_conv1 = bias_variable([CONV1_DEEP])#每一个卷积核一个偏置值

    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool(h_conv1)#进行max-pooling,12-by-40

    #初始化第二个卷积层的权值和偏置
    W_conv2 = weight_variable([CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
    b_conv2 = bias_variable([CONV2_DEEP]) #每一个卷积核一个偏置值

    #把H_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool(h_conv2)#6-by-5

    #23*20的图片第一次卷积后还是23*20,第一次池化后变为12*10
    #第二次卷积后为12*10,第二次池化后变为6*5
    #进过上面操作后得到64张6*5的平面

    #初始化第一全链接层的权值
    W_fc1 = weight_variable([6*5*CONV2_DEEP,FC_SIZE]) #上一层有6*10*64个神经元,全连接层有1024个神经元
    b_fc1 = bias_variable([FC_SIZE])

    #把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool2,[-1,6*5*CONV2_DEEP])
    #求第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #keep_prob用了表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #初始化第二个全连接层
    W_fc2 = weight_variable([FC_SIZE,OUTPUT_NODE])
    b_fc2 = bias_variable([OUTPUT_NODE])

    #计算输出
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    # 结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    #交叉熵代价函数
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    #使用AdamOptimizer进行优化
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
    
#     #自定义损失函数，因为结合位点的标签是[0,1]共有3778，非结合位点的标签是[1,0]有53570，是非平衡数据集，
#     cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction)
#     y1 = tf.argmax(y,1)
#     yshape = tf.shape(y)
#     a = tf.ones([yshape[0]],dtype=tf.int64)
#     loss = tf.reduce_mean( tf.where( tf.greater_equal( y1,a), cross_entropy * LOSS_COEF[1], cross_entropy * LOSS_COEF[0]))

#     #使用AdamOptimizer进行优化
#     train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    #求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(TRAINING_STEPS):
            start = (i * BATCH_SIZE) % x_train_size
            end = min(start + BATCH_SIZE, x_train_size)
            batch_xs = x_train[start:end]
            batch_ys = y_train[start:end]
            sess.run(train_step,feed_dict={x:batch_xs, y: batch_ys, keep_prob: 0.5})
        print(sess.run(accuracy, feed_dict={x:x_test, y:y_test, keep_prob:1.0}))
        pred = sess.run(prediction, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
        return pred


# 绘制混淆矩阵的函数
# 参数1  cm 混淆矩阵中显示的数值 二维数组
# 参数2 cmap 混淆矩阵中的颜色
# 参数3 title 标题
def plot_confusion_matrix(cm, classes, title='混淆矩阵', cmap=plt.cm.Greens):
    # imshow() 表示绘制并显示二维图 有18个参数
    # 参数1 X 混淆矩阵中显示的数值 二维数组
    # 参数2 cmap 颜色 plt.cm.Blues表示蓝色 plt.cm.Reds表示红色 plt.cm.Greens表示绿色
    # 参数5 interpolation 插值法 一般有如下值
    #     nearest 最近邻插值法
    #     bilinear 双线性插值法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.imshow(cm, cmap=cmap, interpolation="nearest")
    plt.title(title)  # 标题
    plt.colorbar()  # 显示颜色的进度条
    tick_marks = np.arange(2)  # [0 1]
    plt.xticks(tick_marks, classes)  # 对x轴上分类进行标记
    plt.yticks(tick_marks, classes)  # 对y轴上分类进行标记

    thresh = np.mean(cm)
    for i in range(2):
        for j in range(2):
            plt.text(i, j, cm[j][i],
                     horizontalalignment='center',
                     color='white' if cm[i][j] >= thresh else 'black')

    plt.xlabel('预测值')
    plt.ylabel('真实值')


    
    
#load benchmark dataset
data = sio.loadmat('PDNA-224-PSSM-Norm-11.mat')

X = data['data']
Y = data['target']





range_data=list(range(DATA_SIZE))    #python3  range函数中需要加list
#print(range_data)

#rind = random.shuffle(range_data)
random.shuffle(range_data)
#print(rind)
#print(range_data)
pred_Y = np.ndarray([DATA_SIZE,OUTPUT_NODE])

X = X.reshape(DATA_SIZE,-1)
X = X[range_data]
Y = Y[range_data]




kf = KFold(n_splits=5)
kf.get_n_splits(X)
#print(X)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #print("len(X_train)={},len(Y_test)={}".format(len(X_train),len(Y_test)))
    pred_Y[test_index] = cnn(X_train,X_test,Y_train,Y_test)

    
    
correct = 0
for i in range(DATA_SIZE):
    if (Y[i] == pred_Y[i]).all():
        correct += 1
print("correct accuracy: {}".format(correct/DATA_SIZE))

Y1 = np.ndarray([DATA_SIZE])
PY = np.ndarray([DATA_SIZE])
for i in range(DATA_SIZE):
    if Y[i][0] == 1:
        Y1[i] = 0
    else:
        Y1[i] = 1
    
    if pred_Y[i][0] == 1:
        PY[i] = 0
    else:
        PY[i] = 1
cnf_matrix = confusion_matrix(Y1, PY)

print(cnf_matrix)
recall = cnf_matrix[1][1] / (cnf_matrix[1][0] + cnf_matrix[1][1])
print('recall: ', recall)
plot_confusion_matrix(cnf_matrix, [0, 1], cmap=plt.cm.Reds)
plt.show()